In [ ]:
method = 'recon'
metric = 'anomaly_score'
std_correction = 'false'
dataset = "bugs"

# Imports
import numpy as np
import seaborn as sns
from skmultiflow.drift_detection.adwin import ADWIN
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm 
import imageio.v2 as imageio
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.cm as cm
import matplotlib.animation as animation
import cv2
from time import sleep
import gc
import glob
import joblib
from joblib import Parallel, delayed
import warnings 
warnings.filterwarnings("ignore")
import os
from random import sample, seed, randint, random, shuffle, choice, choices, uniform, gauss, triangular
from PIL import Image, ImageOps 
from pandas_datareader import data as pdr

In [ ]:
#a = sorted(glob.glob(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/{dataset}/testing/frames/54/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
#a

#adwin = ADWIN()
# Simulating a data stream as a normal distribution of 1's and 0's
#data_stream = np.random.randint(2, size=2000)
# Changing the data concept from index 999 to 2000

#load numpy files from /home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{dataset}/{method}/log/{anomaly_score_total_list.npy, auc.npy, feature_distance_list.npy, psnr_list.npy, labels_list.npy} into variable to analyze

anomaly_score_total_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{dataset}/{method}/log/anomaly_score_total_list.npy", allow_pickle=True)
auc = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{dataset}/{method}/log/auc.npy", allow_pickle=True)
feature_distance_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{dataset}/{method}/log/feature_distance_list.npy", allow_pickle=True)
psnr_list = np.load(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{dataset}/{method}/log/psnr_list.npy", allow_pickle=True)
label_list = np.load(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/frame_labels_{dataset}.npy", allow_pickle=True).tolist()[0]

# normalize python list values to 0-1 range
def normalize_list(list:list):
    min_val = min(list)
    max_val = max(list)
    return [(x-min_val)/(max_val-min_val) for x in list]

def normalize_array(array:np.ndarray):
    min_val = np.min(array)
    max_val = np.max(array)
    return (array-min_val)/(max_val-min_val)

def split_list(list:list, n:int):
    return [list[i:i+n] for i in range(0, len(list), n)]
x = anomaly_score_total_list
print(len(x))
print(len(label_list))
print(auc)
print(feature_distance_list.shape)

In [ ]:
key = list(np.array(psnr_list).reshape(1).tolist()[0].keys())[0]
key

In [ ]:
name = key#'95'
psnr = normalize_list(np.array(psnr_list).reshape(1).tolist()[0][name])
feature_distance = normalize_list(feature_distance_list.tolist()[name])
#psnr = normalize_list(psnr)

#x = normalize_array(x)

adjustment = 4 if method == 'pred' else 0
x = x[adjustment:]
label_list = label_list[adjustment:]
psnr = psnr[adjustment:]
feature_distance = feature_distance[adjustment:]


x_shape = x.shape[0]
start = 0
end = int(x_shape)

if metric == 'anomaly_score':
    x = x[start:end]
elif metric == 'feature_distance':
    x = feature_distance[start:end]
elif metric == 'psnr':
    x = psnr[start:end]


In [ ]:
len(feature_distance)

In [ ]:
label_list

In [ ]:

label_list = label_list[start:end]

In [ ]:
label_points = {'x':[], 'y':[]}
label_plot = {'x':[], 'y':[]}
for i in range(len(label_list)):
    if label_list[i] == 1:
        label_points['x'].append(i)
        label_points['y'].append(x[i])
        
        label_plot['y'].append(x[i])
    else:
        label_plot['y'].append(0)
    label_plot['x'].append(i)

In [ ]:
# use adwin for change detection of the timeseries represented by x
adwin = ADWIN(.05)
drift = {'x': [], 'y': []}
xpos = 0
for i in range(x_shape):  # number of frames in vide0
    g = adwin.add_element(x[i])
    if adwin.detected_change():
        drift['y'].append(x[i])
        drift['x'].append(xpos)
        adwin.reset()
    elif adwin.detected_warning_zone():
        drift['y'].append(x[i])
        drift['x'].append(xpos)

    xpos += 1
    

In [ ]:

# plot the changes
fig, ax = plt.subplots()
fig.set_size_inches(200.5, 7.5, forward=True)
plt.xticks(np.arange(0, end+1, 10.0))

ax.plot(np.arange(len(x)), x, color='black', label='anomaly score', linewidth=0.6)
ax.scatter(np.arange(len(x)), x, color='black', label='anomaly score', s=4)
ax.scatter(label_points['x'], label_points['y'], color='red', marker='x', s=10)
# ax.scatter(drift['x'], drift['y'],color='green', marker='|', s=1000000)


ax.legend([metric,metric,'ground_truth'])
plt.savefig(f"/home/smoothjazzuser/Desktop/{dataset}_{method}_{metric}_folder-{name}_std_correction-{std_correction}.svg")
plt.show()



In [ ]:
class timeseries_anomaly_detection:
    def __init__(self, stock=False, start=False, end=False, x=False, y=False) -> None:
        if stock:
            self.download_data(stock, start, end)
        else:
            self.data = x
            self.data = self.normalize_data(self.data)
            self.label_points = y
            self.data = pd.Series(self.data)
            self.label_points = pd.DataFrame.from_dict(self.label_points, orient='index')
            self.label_points = self.label_points.transpose() 
            self.zeros = pd.DataFrame(np.zeros(self.data.shape[0]), columns=['zeros'])

    def normalize_data(self, data):
        #return data
        return (data - data.min()) / (data.max() - data.min())

    def download_data(self, ticker, start_date, end_date):
        self.data = pdr.get_data_yahoo(ticker, start_date, end_date)
        self.data = self.data.dropna()
        print(self.data.head())
        #normalize between 0 and 1
        self.data = self.normalize_data(self.data['Adj Close'])
        return self.data

    def rolling_average(self, data, window):
        roll = data.rolling(window).mean()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return  roll

    def rolling_std(self, data, window):
        roll = data.rolling(window).std()
        #normalize between 0 and 1
        roll = self.normalize_data(roll)
        # correct for window size shift
        #roll = roll.shift(-window)
        return roll

    def gradient_at_each_point(self, data, window=False):
        if window: 
            self.gradient = np.gradient(data.rolling(window))
        else:
            self.gradient = np.gradient(self.data)
        self.gradient = pd.Series(self.gradient)
        #return gradient

    def points_above_rollingaverage_and_bellow_rollingstd(self, data, window, rolling=False):
        if rolling:
            rolling_mean = self.rolling_average(data, window)
            self.gradient_at_each_point(rolling_mean, window)
            rolling_std = self.rolling_std(data, window)
            points = data[(data > rolling_mean) & (data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        else:
            rolling_std = self.rolling_std(data, window)
            self.gradient_at_each_point(data, False)
            points = data[(data < rolling_std) & (self.gradient <= self.zeros.zeros)]
        return points

    def plot(self, window=20, alpha=0.5, plots = ['data', 'mean', 'std', 'exponential', 'troughs', 'bet_std_mean', 'gradient']):
        plt.figure(figsize=(int(len(self.data)/76*1.5), 7))

        if 'mean' in plots:
            rolling_mean = self.rolling_average(self.data, window)
            plt.plot(rolling_mean, label='Rolling Mean')
        if 'std' in plots:
            rolling_std = self.rolling_std(self.data, window)
            plt.plot(rolling_std, label='Rolling Std', color='green')
        if 'exponential' in plots:
            exponential_smoothing = self.exponential_smoothing(self.data, alpha)
            plt.plot(exponential_smoothing, label='Exponential Smoothing')
        if 'troughs' in plots:
            troughs = self.lines_at_troughs(self.data, window)
            plt.plot(troughs, label='Lines at Troughs')
        if 'bet_std_mean' in plots:
            self.points = self.points_above_rollingaverage_and_bellow_rollingstd(self.data, window, rolling=False)
            # point y values should equal to data y values
            plt.plot(self.points, 'ro', label='Flagged points', color='purple', markersize=10, alpha=0.5, marker='o')
            # plot these points with y-values = data y values
        if 'data' in plots:
            plt.plot(np.arange(len(self.data)), self.data, color='black', label='data', linewidth=0.6)
            plt.scatter(np.arange(len(self.data)), self.data, color='black', label='data', s=4)
            plt.scatter(self.label_points['x'], self.label_points['y'], color='red', marker='x', s=10)
        
        plt.legend(loc='upper left')
        #return all plots
        self.all_plots = plt.gcf()
        #del plot
        plt.close('all')
        return self.all_plots
    
    def show_plot(self):
        return self.all_plots

series = timeseries_anomaly_detection(x=x, y=label_points)


In [ ]:
series.plot(window=10, alpha=0.5,  plots = ['bet_std_mean', 'std', 'data','mean'])

In [ ]:
series.plot(window=10, alpha=0.5,  plots = ['bet_std_mean', 'std','mean'])

In [ ]:
def load_(filename):
    i = plt.imread(filename)
    return  i

r = int((len(label_list)-1)/1)
#preds = []
preds = sorted(glob.glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{dataset}/{method}/log/preds/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))
diffs = sorted(glob.glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/exp/{dataset}/{method}/log/diffs/*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0]))

ground_truth = []
folders = sorted(glob.glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2]))
files = [sorted(glob.glob(f"{file}*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0])) for file in folders]

for folder in sorted(glob.glob(f"/home/smoothjazzuser/Desktop/videogame-anomoly/MNAD/dataset/{dataset}/testing/frames/*/"), key=lambda x: int(x.split('/')[-2])):
    for file in sorted(glob.glob(f"{folder}*.jpg"), key=lambda x: int(x.split('/')[-1].split('.')[0])):
        ground_truth.append(plt.imread(file))

#ground_truth = Parallel(n_jobs=8)(delayed(load_)(file) for file in tqdm(files))
#preds = Parallel(n_jobs=8)(delayed(load)(file) for file in tqdm(preds))
#diffs = Parallel(n_jobs=8)(delayed(load)(file) for file in tqdm(diffs))


In [ ]:
preds = [plt.imread(pred) for pred in preds]
diffs = [plt.imread(diff) for diff in diffs]

In [ ]:
!rm -rf /tmp/*.jpg
!rm -rf /tmp/*.jpg
!rm -rf /tmp/*.svg

In [ ]:

def plot_images(i):
    window = 50
    if i > 50:
        window = i - 50
    else:
        window = 0
    
    #plot preds
    fig = plt.figure()
    fig.set_size_inches(10, 10, forward=True)
    ax = fig.add_subplot(2,2,1)
    ax.set_title(f"prediction {window}:{i}")
    ax.imshow(preds[i], cmap='gray') 

    #plot diffs
    ax = fig.add_subplot(2,2,2)
    ax.set_title(f"diff {window}:{i}")
    ax.imshow(diffs[i], cmap='gray')

    #plot ground_truth
    ax = fig.add_subplot(2,2,4)
    ax.set_title(f"ground truth {window}:{i}")
    ax.imshow(ground_truth[i], cmap='gray')

    #plot anomaly score
    ax = fig.add_subplot(2,2,3)
    ax.set_title(f"anomaly score {window}:{i}")
    ax.plot(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', linewidth=0.6)
    ax.scatter(np.arange(len(x[window:i])), x[window:i], color='black', label='anomaly score', s=4)
    ax.scatter(np.arange(len(label_plot['x'][window:i])), label_plot['y'][window:i], color='red', marker='x', s=4)

    # find all search.points that are in the window
    matching = [(i,series.points[x]) for i, x in enumerate(np.arange(window, i)) if x in series.points.index]
    # plot these points with y-values = data y values
    ax.scatter([x[0] for x in matching], [x[1] for x in matching], label='Flagged points', color='purple', s=10, alpha=0.5, marker='o')


    ax.legend([metric])
    fig.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0)
    #set ax to show 50 x values
    ax.set_xlim(0, 50)
    ax.set_ylim(0, 1)
    fig.savefig(f"/tmp/{i}.jpg", bbox_inches='tight', pad_inches=0)
    
    fig.clear()
    plt.close(fig)
    del ax
    del window
    del fig
    return None

#split r into 10 chunks
rr = list(range(r))

a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i) for i in rr[0:int(len(rr)/2)]) #multiprocessing'  'loky', require='sharedmem'
del a
gc.collect()
a = Parallel(n_jobs=12, backend='multiprocessing')(delayed(plot_images)(i) for i in rr[int(len(rr)/2):-1])
del a
gc.collect()


del preds
del diffs
del ground_truth
gc.collect()


In [ ]:
a = imageio.imread(f"/tmp/0.jpg").shape
h,w = a[0], a[1]
def load_frames(i):
    return np.uint8(cv2.resize(imageio.imread(f"/tmp/{i}.jpg"), (h,w)))

frames = Parallel(n_jobs=12, backend='multiprocessing')(delayed(load_frames)(i) for i in tqdm(range(r-1))) 


In [ ]:
!rm -rf /tmp/*.jpg
!rm -rf /tmp/*.jpg
!rm -rf /tmp/*.svg
#delete all variables except frames
gc.collect()

In [ ]:
# save frames as an mp4
#resize all images in frames to the same size (the largest common size)
#get max size of image in frames

#h = max([frame.shape[0] for frame in frames])
#w = max([frame.shape[1] for frame in frames])
#frames = [cv2.resize(frame, (w,h)) for frame in frames]
imageio.mimsave(f'/home/smoothjazzuser/Desktop/{dataset}_{method}_{metric}_folder-{name}_std_correction-{std_correction}.mp4', frames, fps=10)

In [ ]:
#free all variables
del frames
gc.collect()